In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import librosa
import math
import json 
import matplotlib.pyplot as plt
import IPython
from random import randint
import math

#ml packages
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models, Sequential
import statistics

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
path = '/kaggle/input/moroccan-music-data/MoroccanMusic'

genre_dict = {"gnawa":0,"chaabi":1,"andalusian":2, "rai":3, "imazighn":4, "rap":5 }



**Data Preprocessing + feature exctractiong**

In [8]:
def preprocess(dataset_path, num_mfcc=40, n_fft=2048, hop_length=512, num_segment=10):
    data = {'audio_path':[], "labels": [], "mfcc": []}
    sample_rate = 22050
    samples_per_segment = int(sample_rate * 30 / num_segment)
    
    for label_idx, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):
        if dirpath == dataset_path:
            continue
        dirname = os.path.basename(dirpath)
        print(dirname)
        lbl = genre_dict[dirname]
        for f in sorted(filenames):
            if not f.endswith('.wav'):
                continue
            if(f=="jazz.00054.wav"):
            # As librosa only read files <1Mb
              continue
            file_path = os.path.join(dirpath, f)
            
            try:
                y, sr = librosa.load(file_path, sr=sample_rate)
            except:
                continue
            try:
              for n in range(num_segment):
                 start_sample = samples_per_segment * n
                 end_sample = start_sample + samples_per_segment
                
                 mfcc = librosa.feature.mfcc(y=y[start_sample:end_sample], sr=sr, n_mfcc=40, n_fft=2048, hop_length = 512)

                 mfcc = mfcc.T
                 if len(mfcc) == math.ceil(samples_per_segment / hop_length):
                     data["audio_path"].append(file_path)
                     data["mfcc"].append(mfcc.tolist())
                     data["labels"].append(lbl)
            except:
                print(file_path + str(lbl))
                
      
    return data



In [9]:
data = preprocess(path)

rap


/opt/conda/lib/python3.10/site-packages/librosa/core/spectrum.py:256: UserWarning: n_fft=2048 is too large for input signal of length=0
  warnings.warn(


rai
chaabi
gnawa
imazighn
andalusian


In [10]:
data["audio_path"][0], data["labels"][0] 

('/kaggle/input/moroccan-music-data/MoroccanMusic/rap/rap.00000.wav', 5)

**Save preprocessed data to disk**

In [ ]:
data_df = pd.DataFrame(data)
data_df.to_csv('/kaggle/working/moroccanMusic_extracted_features.csv')

In [4]:
data = pd.read_csv('/kaggle/input/temp-files/moroccanMusic_extracted_features.csv')

**Ml part**

In [11]:
X = np.array(data["mfcc"], dtype=object)
X = np.asarray(X).astype('float32')
y = np.array(data["labels"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:
y_train[0]

In [12]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5])

In [13]:
input_shape= (X_train.shape[1],X_train.shape[2])
input_shape

(130, 40)

**Model**

In [14]:
classifier = Sequential()
classifier.add(layers.LSTM(64, input_shape=(input_shape[0], input_shape[1]), return_sequences=True))
classifier.add(layers.LSTM(64))
classifier.add(layers.Dense(64, activation='relu'))
classifier.add(layers.Dense(10, activation='softmax')) 


In [ ]:
import tensorflow as tf

In [15]:
optimizer = tf.keras.optimizers.Adam(lr=0.001)
classifier.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.summary()
classifier.fit(X_train, y_train, validation_data=(X_val,y_val), batch_size=32, epochs=60, verbose=2)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 64)           26880     
                                                                 
 lstm_1 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 64,714
Trainable params: 64,714
Non-trainable params: 0
_________________________________________________________________
Epoch 1/60
110/110 - 21s - loss: 1.4247 - accuracy: 0.4811 - val_loss: 1.0934 - val_accuracy: 0.6144 - 21s/epoch - 195ms/step
Epoch 2/60
110/110 - 16s - loss: 0.9034 - accuracy: 0.6731

In [16]:
train_loss, train_acc  = classifier.evaluate(X_train, y_train, batch_size=128)

28/28 [==============================] - 4s 148ms/step - loss: 9.2864e-05 - accuracy: 1.0000


In [17]:
test_loss, test_acc  = classifier.evaluate(X_test, y_test, batch_size=128)

12/12 [==============================] - 2s 137ms/step - loss: 0.2449 - accuracy: 0.9636


In [18]:
number_to_genre_dict = {0:"gnawa" , 1:"chaabi", 2:"andalusian", 3:"rai", 4:"imazighn", 5:"rap"}

**single predictions examples**

In [19]:
def print_class_name(classes):
    print(f'the predicted class is {number_to_genre_dict[statistics.mode(classes)]}')    

In [20]:
def class_pred(classifier, file):  
    y, sr = librosa.load(file)
    oneSong = []
    for n in range(10):
        start_sample = 22050*3  * n
        end_sample = start_sample + 22050*3
                
        mfcc = librosa.feature.mfcc(y=y[start_sample:end_sample], sr=sr, n_mfcc=40, n_fft=2048, hop_length = 512)

        mfcc = mfcc.T
        
        if len(mfcc) == math.ceil( 22050*3 / 512 ):
                oneSong.append(mfcc.tolist())



    oneSong = np.array(oneSong, dtype=object)
    oneSong = nsong = np.asarray(oneSong).astype('float32')
    oneSong.shape

    prediction = classifier.predict(oneSong)

    classes_x =np.argmax(prediction,axis=1)
    return classes_x

In [29]:
myfile = '/kaggle/input/temp-files/Bob_MarleyEverythings_Gonna_Be_Alright.wav'
print_class_name(class_pred(classifier, myfile))

1/1 [==============================] - 0s 66ms/step
the predicted class is gnawa


**Save/load the model to/from disk**

In [23]:
# serialize model to JSON
model_json = classifier.to_json()
with open("Moroccanmodel.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("Moroccanmodel.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('/kaggle/working/Moroccanmodel.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/kaggle/working/Moroccanmodel.h5")
print("Loaded model from disk")